This notebook shows sprinkling result in SR1 data, with Rn220 beta events sprinkled.

Lanqing, Mar 15 2024

# Preparation

In [ ]:
import saltax
import straxen
import cutax
from tqdm import tqdm
import gc 
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from itertools import cycle
from saltax.match.utils import *
straxen.print_versions()

In [ ]:
sr1 = '051613,050996,051948,051939,051910,051909,051905,047876,047865,047860,053167,053153,053139,052997,049123,049450,049505,049521,049544'.split(',')
modes = ['tpc_radon222', 'tpc_radon222',
         'ambe_linked_hev', 'ambe_linked_hev', 'ambe_linked_hev', 'ambe_linked_hev', 'ambe_linked_hev',
         'ybe_linked', 'ybe_linked', 'ybe_linked', 
         'tpc_kr83m', 'tpc_kr83m', 'tpc_kr83m', 'tpc_kr83m',
         'tpc_radon_hev', 'tpc_radon_hev', 'tpc_radon_hev', 'tpc_radon_hev', 'tpc_radon_hev']
modes_dict = {}
for i in range(len(sr1)):
    modes_dict[sr1[i]] = modes[i]

In [ ]:
st_salt = saltax.contexts.sxenonnt(runid=51613, # put anything here should work
             saltax_mode='salt',
             output_folder='/scratch/midway2/yuanlq/salt/beta',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')
st_simu = saltax.contexts.sxenonnt(runid=51613, # put anything here should work
             saltax_mode='simu',
             output_folder='/scratch/midway2/yuanlq/salt/beta',
             faxconf_version="sr0_v4",
             generator_name='flat',
             recoil=8,
             mode='all')

In [ ]:
runs_available = []
for run in sr1:
    print(run, modes_dict[run], st_salt.is_stored(run, 'event_info'), st_simu.is_stored(run, 'event_info'))

print('==========')
for run in sr1:
    if st_salt.is_stored(run, 'event_info') and st_simu.is_stored(run, 'event_info'):
        print(run, modes_dict[run], 
              st_salt.is_stored(run, 'cuts_basic'), 
              st_simu.is_stored(run, 'cuts_basic'))

# Let's sprinkle

## SR1 AmBe

In [ ]:
(events_simu, 
 events_salt, 
 truth, 
 match, 
 events_salt_matched_to_simu, 
 events_simu_matched_to_salt
) = load_events(['051939', '051910', '051909', '051905'], st_salt, st_simu);

In [ ]:
get_single_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
get_n_minus_1_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                  n_bins = 16, title='Without Cuts Ambience Interference in SR1 AmBe HEV')
compare_templates(events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu,
                                                              AmBe_CUTS_EXCEPT_S2PatternS1Width)], 
                  events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt,
                                                              AmBe_CUTS_EXCEPT_S2PatternS1Width)], 
                  n_bins = 16, title='With Cuts Ambience Interference in SR1 AmBe HEV')

Those below are effect of ambience interference + cut acceptance on clean simulated events.

In [ ]:
_ambe_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
            n_bins=16, coord = 's1_area', plot=True,
            indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 AmBe')
_ambe_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
            n_bins=16, coord = 's2_area', plot=True,
            indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 AmBe')
_ambe_z_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
            n_bins=16, coord = 'z', plot=True,
            indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 AmBe')

In [ ]:
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt,
                                AmBe_CUTS_EXCEPT_S2PatternS1Width)
ambe_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's1_area', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 AmBe')
ambe_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's2_area', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 AmBe')
ambe_z_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = AmBe_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 'z', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 AmBe')

In [ ]:
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu,
                                AmBe_CUTS_EXCEPT_S2PatternS1Width)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 AmBe HEV',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 AmBe HEV', 
               coord='s1_area', s1s2='S1', n_bins=16, ylim=(-2,10))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 AmBe HEV', 
               coord='s2_area', s1s2='S2', n_bins=16, ylim=(-2,100))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 AmBe HEV', 
               coord='z', s1s2='S2', n_bins=16, ylim=(-2,100))

## SR1 YBe

In [ ]:
(events_simu, 
 events_salt, 
 truth, 
 match, 
 events_salt_matched_to_simu, 
 events_simu_matched_to_salt
) = load_events(['047876', '047865', '047860'], st_salt, st_simu);

In [ ]:
get_single_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
get_n_minus_1_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                  n_bins = 16, title='Without Cuts Ambience Interference in SR1 YBe')
compare_templates(events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  n_bins = 16, title='With Cuts Ambience Interference in SR1 YBe')

Why so clean? This makes no sense

In [ ]:
_ybe_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's1_area', plot=True,
                              indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 YBe',
                              bbox_to_anchor=(0.5, 1.52))
_ybe_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's2_area', plot=True,
                              indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 YBe',
                              bbox_to_anchor=(0.5, 1.52))
_ybe_z_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 'z', plot=True,
                             indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 YBe',
                             bbox_to_anchor=(0.5, 1.52))

In [ ]:
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
ybe_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                             all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 's1_area', plot=True,
                             indv_cut_type='n_minus_1', 
                             title='Conditional N-1 Cut Acceptance Measured in SR1 YBe',
                             bbox_to_anchor=(0.5, 1.52))
ybe_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                          all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                          n_bins=16, coord = 's2_area', plot=True,
                          indv_cut_type='n_minus_1', 
                          title='Conditional N-1 Cut Acceptance Measured in SR1 YBe',
                          bbox_to_anchor=(0.5, 1.52))
ybe_z_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                            all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                            n_bins=16, coord = 'z', plot=True,
                            indv_cut_type='n_minus_1', 
                            title='Conditional N-1 Cut Acceptance Measured in SR1 YBe',
                            bbox_to_anchor=(0.5, 1.52))

In [ ]:
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 YBe',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 YBe',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 YBe',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 YBe',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 YBe',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 YBe', 
               coord='s1_area', s1s2='S1', n_bins=16, ylim=(-2,10))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 YBe', 
               coord='s2_area', s1s2='S2', n_bins=16, ylim=(-2,100))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 YBe', 
               coord='z', s1s2='S2', n_bins=16, ylim=(-2,100))

## SR1 Kr83m

In [ ]:
(events_simu, 
 events_salt, 
 truth, 
 match, 
 events_salt_matched_to_simu, 
 events_simu_matched_to_salt
) = load_events(['053167', '053153', '053139', '052997'], st_salt, st_simu);

In [ ]:
get_single_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)
get_n_minus_1_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                  n_bins = 16, title='Without Cuts Ambience Interference in SR1 Kr83m')
compare_templates(events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  n_bins = 16, title='With Cuts Ambience Interference in SR1 Kr83m')

In [ ]:
_kr_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 's1_area', plot=True,
                             indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Kr83m')
_kr_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 's2_area', plot=True,
                             indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Kr83m')
_kr_z_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                            n_bins=16, coord = 'z', plot=True,
                            indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Kr83m')

In [ ]:
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
ambe_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's1_area', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220')
ambe_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 's2_area', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220')
ambe_z_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                              all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                              n_bins=16, coord = 'z', plot=True,
                              indv_cut_type='n_minus_1', 
                              title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220')

In [ ]:
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
kr_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                            all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                            n_bins=16, coord = 's1_area', plot=True,
                            indv_cut_type='n_minus_1', 
                            title='Conditional N-1 Cut Acceptance Measured in SR1 Kr83m',
                            bbox_to_anchor=(0.5, 1.52))
kr_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                         all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                         n_bins=16, coord = 's2_area', plot=True,
                         indv_cut_type='n_minus_1', 
                         title='Conditional N-1 Cut Acceptance Measured in SR1 Kr83m',
                         bbox_to_anchor=(0.5, 1.52))
kr_z_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                           all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                           n_bins=16, coord = 'z', plot=True,
                           indv_cut_type='n_minus_1', 
                           title='Conditional N-1 Cut Acceptance Measured in SR1 Kr83m',
                           bbox_to_anchor=(0.5, 1.52))

In [ ]:
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Kr83m',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Kr83m',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Kr83m',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Kr83m',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Kr83m',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Kr83m', 
               coord='s1_area', s1s2='S1', n_bins=16, ylim=(-2,10))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Kr83m', 
               coord='s2_area', s1s2='S2', n_bins=16, ylim=(-2,100))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Kr83m', 
               coord='z', s1s2='S2', n_bins=16, ylim=(-2,100))

## SR1 Rn220

In [ ]:
(events_simu, 
 events_salt, 
 truth, 
 match, 
 events_salt_matched_to_simu, 
 events_simu_matched_to_salt
) = load_events(['049450'], st_salt, st_simu);

In [ ]:
get_single_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)
get_n_minus_1_cut_acc(events_salt_matched_to_simu, events_simu_matched_to_salt,
                      all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width)

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                  n_bins = 16, title='Without Cuts Ambience Interference in SR1 Rn220')
compare_templates(events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  n_bins = 16, title='With Cuts Ambience Interference in SR1 Rn220')

In [ ]:
compare_templates(events_salt_matched_to_simu, events_simu_matched_to_salt, 
                  n_bins = 16, title='Without Cuts Ambience Interference in SR1 Rn220')
compare_templates(events_salt_matched_to_simu[apply_cut_lists(events_salt_matched_to_simu,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  events_simu_matched_to_salt[apply_cut_lists(events_simu_matched_to_salt,
                                                              ALL_CUTS_EXCEPT_S2PatternS1Width)], 
                  n_bins = 16, title='With Cuts Ambience Interference in SR1 Rn220')

In [ ]:
_rn220_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 's1_area', plot=True,
                             indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Rn220',
                             bbox_to_anchor=(0.5, 1.52))
_rn220_s2_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                             n_bins=16, coord = 's2_area', plot=True,
                             indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Rn220',
                             bbox_to_anchor=(0.5, 1.52))
_rn220_z_cut_eff = get_cut_eff(events_salt_matched_to_simu, all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                            n_bins=16, coord = 'z', plot=True,
                            indv_cut_type='n_minus_1', title='N-1 Cut Acceptance Measured in SR1 Rn220',
                            bbox_to_anchor=(0.5, 1.52))

In [ ]:
mask_simu_cut = apply_cut_lists(events_simu_matched_to_salt,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
rn220_s1_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                            all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                            n_bins=16, coord = 's1_area', plot=True,
                            indv_cut_type='n_minus_1', 
                            title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220',
                            bbox_to_anchor=(0.5, 1.52))
rn220_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                         all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                         n_bins=16, coord = 's2_area', plot=True,
                         indv_cut_type='n_minus_1', 
                         title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220',
                         bbox_to_anchor=(0.5, 1.52))
rn220_z_cut_eff = get_cut_eff(events_salt_matched_to_simu[mask_simu_cut], 
                           all_cut_list = ALL_CUTS_EXCEPT_S2PatternS1Width, 
                           n_bins=16, coord = 'z', plot=True,
                           indv_cut_type='n_minus_1', 
                           title='Conditional N-1 Cut Acceptance Measured in SR1 Rn220',
                           bbox_to_anchor=(0.5, 1.52))

In [ ]:
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Rn220',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Rn220',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu, 
              events_simu_matched_to_salt, 
              title='Before Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
mask_salt_cut = apply_cut_lists(events_salt_matched_to_simu,
                                ALL_CUTS_EXCEPT_S2PatternS1Width)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Rn220',
              coords=['z', 's2_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Rn220',
              coords=['z', 's2_range_90p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_rise_time'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_range_50p_area'], n_bins=16)
compare_bands(events_salt_matched_to_simu[mask_salt_cut], 
              events_simu_matched_to_salt[mask_simu_cut], 
              title='After Cuts Ambience Interference in SR1 Rn220',
              coords=['s1_area', 's1_range_90p_area'], n_bins=16)

In [ ]:
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Rn220', 
               coord='s1_area', s1s2='S1', n_bins=16, ylim=(-2,10))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Rn220', 
               coord='s2_area', s1s2='S2', n_bins=16, ylim=(-2,100))
show_area_bias(events_salt_matched_to_simu[mask_salt_cut&mask_simu_cut], 
               events_simu_matched_to_salt[mask_salt_cut&mask_simu_cut], 
               title='After Cuts Ambience Interference in SR1 Rn220', 
               coord='z', s1s2='S2', n_bins=16, ylim=(-2,100))